Load some utils function + annotations first

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import json
from utils import load_image_annotations, display_image_with_keypoints

In [ ]:
annotations = json.load(open("/root/data/gtsf_2.0/registration_test/gtsf_in_air_labels.json"))

load reference image

In [ ]:
static_image, static_mask, static_keypoints, static_kp_map = load_image_annotations(annotations[3], new_shape=(800, 600))

In [ ]:
display_image_with_keypoints(static_image, static_keypoints)

now looooop

In [ ]:
from copy import copy

from dipy.viz import regtools

from utils import translate_moving, register, display_warped_keypoints, display_pairs_with_keypoints, calculate_errors
from utils import display_keypoints_gt_pred

In [ ]:
display = False

In [ ]:
for annotation in annotations[2:]:
    moving_image, moving_mask, moving_keypoints, moving_kp_map = load_image_annotations(annotation, 
                                                                                        new_shape=(800, 600))
    
    # display 
    if display:
        display_image_with_keypoints(moving_image, moving_keypoints)
        regtools.overlay_images(static_mask, moving_mask, 'Static', 'Overlay', 'Moving', 'input_images.png')
    
    # translate
    moving_translated, translation_vector = translate_moving(static_mask, moving_mask)
    if display:
        regtools.overlay_images(static_mask, moving_translated, 'Static', 'Overlay', 'Moving', 'input_images.png')
    
    # register
    mapping = register(static_mask, moving_translated)
    
    # plot deformation field
#     regtools.plot_2d_diffeomorphic_map(mapping, 10, 'diffeomorphic_map.png')
    
    # forward stuff
    warped_moving = mapping.transform(moving_translated, 'linear')
    if display:
        regtools.overlay_images(static_mask, warped_moving, 'Static','Overlay','Warped moving',
                                'direct_warp_result.png')
    
    # backward stuff
    warped_static = mapping.transform_inverse(static_mask, 'linear')
    if display:
        regtools.overlay_images(warped_static, moving_translated,'Warped static','Overlay','Moving',
                                'inverse_warp_result.png')
    
    # warped keypoints
    warped_kp_map = mapping.transform_inverse(static_kp_map, "linear")
    if display:
        display_warped_keypoints(moving_image, warped_kp_map, translation_vector)
    
    # display_pairs_with_keypoints(moving_image, moving_kp_map, warped_kp_map, translation_vector)
    display_keypoints_gt_pred(moving_image, moving_kp_map, warped_kp_map, translation_vector)
    
    calculate_errors(moving_keypoints, warped_kp_map, translation_vector)
    print("#"*100)
    print("#"*100)
    print("#"*100)
